# Strip Font Encryption from DRM-Protected EPUBs

**Purpose:** Remove font encryption from 8 books while preserving all text content

**What we discovered:** The DRM only encrypts fonts - all teaching text is already readable!
We just need to rebuild the EPUBs without the encrypted fonts.

In [ ]:
import ebooklib
from ebooklib import epub
from pathlib import Path
import zipfile
import shutil

print("✓ Libraries loaded")

In [ ]:
# Configuration
EPUB_DIR = Path("epub_directory/epub_directory")

DRM_FILES = [
    "How-to-Solve-Our-Human-Problems-US.epub",
    "How_to_Transform_Your_Life-US.epub",
    "Modern-Buddhism-2nd-US-V2.epub",
    "Modern-Buddhism-2nd-US-V3.epub",
    "Modern-Buddhism-2nd-US-v1.epub",
    "The New Eight Steps to Happiness - eBook ePUB format.epub",
    "The_New_Meditation_Handbook-US.epub",
    "the-oral-instructions-of-mahamudra-en-2nd.epub"
]

print(f"Directory: {EPUB_DIR}")
print(f"Files: {len(DRM_FILES)}")

In [ ]:
def strip_font_drm_simple(input_path, output_path):
    """
    Simple approach: Copy EPUB as ZIP, remove encrypted font files.
    This preserves all structure including NCX/navigation.
    """
    import tempfile
    import os
    
    try:
        # Create temp directory
        with tempfile.TemporaryDirectory() as tmpdir:
            tmpdir_path = Path(tmpdir)
            
            # Extract EPUB (it's a ZIP file)
            with zipfile.ZipFile(input_path, 'r') as zip_ref:
                zip_ref.extractall(tmpdir_path)
            
            # Find and remove encrypted fonts
            fonts_removed = 0
            for font_file in tmpdir_path.rglob('*.otf'):
                font_file.unlink()
                fonts_removed += 1
            for font_file in tmpdir_path.rglob('*.ttf'):
                font_file.unlink()
                fonts_removed += 1
            
            # Remove encryption.xml
            enc_file = tmpdir_path / 'META-INF' / 'encryption.xml'
            if enc_file.exists():
                enc_file.unlink()
            
            # Repackage as EPUB
            with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as epub_zip:
                # Must write mimetype first, uncompressed
                mimetype_file = tmpdir_path / 'mimetype'
                if mimetype_file.exists():
                    epub_zip.write(mimetype_file, 'mimetype', compress_type=zipfile.ZIP_STORED)
                
                # Write everything else
                for file in tmpdir_path.rglob('*'):
                    if file.is_file() and file.name != 'mimetype':
                        arcname = file.relative_to(tmpdir_path)
                        epub_zip.write(file, arcname)
            
            return True, f"Removed {fonts_removed} font files"
            
    except Exception as e:
        return False, str(e)

print("✓ Function defined")

In [ ]:
print("=" * 80)
print("STRIPPING FONT ENCRYPTION")
print("=" * 80)
print()

results = []

for i, filename in enumerate(DRM_FILES, 1):
    input_path = EPUB_DIR / filename
    output_filename = filename.replace('.epub', '_clean.epub')
    output_path = EPUB_DIR / output_filename
    
    print(f"[{i}/{len(DRM_FILES)}] {filename}")
    
    if not input_path.exists():
        print(f"  ⚠ Not found")
        results.append((filename, False, "Not found"))
        continue
    
    success, message = strip_font_drm_simple(input_path, output_path)
    
    if success:
        print(f"  ✓ {message}")
        print(f"  → {output_filename}")
        results.append((filename, True, message))
    else:
        print(f"  ✗ {message}")
        results.append((filename, False, message))
    print()

success_count = sum(1 for _, s, _ in results if s)
print("=" * 80)
print(f"Done: {success_count}/{len(DRM_FILES)} files")
print(f"Location: {EPUB_DIR}")
print("=" * 80)